# Генераторы

_Генератором_ в Python называется некоторый _итерируемый_ объект.
Это мало о чём говорит.
Рассмотрим этот вопрос подробно, так как генераторы лежат в основе такой библиотеки дискретно-событийного моделирования, как [SimPy](https://simpy.readthedocs.io/en/latest/index.html).
Поэтому генераторы необходимо освоить, чтобы легко работать с дискретно-событийным программированием.
Более того, поняв генераторы, вы легче поймёте асинхронное программирование, которое широко применяется в Web-разработке.

Разбираться начнём с примеров генераторов стандартной библиотеки.
Затем перейдём к созданию собственных генераторов.

## Что такое генератор

_Генератор_ - это объект, возвращаемый _генераторной функцией_.

_Генераторная функция_ - это функция, в которой есть оператор `yield`, который и отвечает за возвращение объекта генератора.
Пример:

In [1]:
def gen_func():
    yield 1

g = gen_func()
g

<generator object gen_func at 0x7fea9f399700>

Чтобы получить значение из генератора, необходимо знать, что генератор поддерживает протокол итератора, т.е. к нему можно применить функцию `next`:

In [2]:
next(g)

1

Как известно, когда итератор достигает своего конца, он инициирует исключение `StopIteration`.
То же самое случится, если мы ещё раз применим `next` к генератору `g`:

In [3]:
from termcolor import cprint

try:
    next(g)
except StopIteration as ex:
    cprint("StopIteration:", "red", end=" ")
    print("генератор пуст")

StopIteration: генератор пуст


Но раз генератор поддерживает протокол итератора, значит, он может использоваться в цикле `for`, который этот протокол реализует:

In [4]:
for i in gen_func():
    print(i)

1


Таковы основы, однако созданный генератор имеет мало смысла.
Создадим, например, генератор, возвращающий последовательно числа от 0 до заданного `n`:

In [5]:
def first_n(n):
    counter = 0
    while counter < n:
        # Генерируем значение
        yield counter
        # Увеличиваем счётчик (движемся по циклу)
        counter += 1

Теперь выведем первые 5 неотрицательных чисел:

In [6]:
for i in first_n(5):
    print(i, end=" ")

0 1 2 3 4 

Кое-что напоминает?
К примеру, это:

In [7]:
for i in range(5):
    print(i, end=" ")

0 1 2 3 4 

Разберём по шагам, что происходило в этом цикле:

```{code}
for i in first_n(5):
    print(i, end=" ")
```

1. Вызвана генеративная функция `first_n(5)`, вернувшая объект генератора:

In [8]:
g = first_n(5)
g

<generator object first_n at 0x7fea9f39a030>

При этом внутри `first_n` инициализировани счётчик `counter = 0` и цикл `while`.
Дойдя до `yield`, поток выполнения вернулся в точку вызова `first_n`, т.е. в следующую за `g = first_n(...)` строку.

2. К генератору применена функция `next`. Поток упраления переходит в функцию `first_n` к оператору `yield`, который генерирует текущее значение `counter`. Цикл в `first_n`, повторяясь, и снова доходит до оператора `yield`. Поток управления возвращается в место вызова `next`. Результат сработавшего `yield` сохраняется в счётчике `i`:

In [9]:
i = next(g)

3. Вывод на экран значения `i`:

In [10]:
print(i)

0


4. Цикл повторяется с п.2:

In [11]:
# Итерация 2
i = next(g)
print(i)
# 3...
i = next(g)
print(i)
# И т.д., пока не закончатся значения в генераторе

1
2


Когда генератор будет исчерпан, инициируется исключение `StopIteration`, говорящее о завершении протокола итератора.

## Генераторное выражение

Есть ещё один способ создания генератора - _генеративное (генераторное) выражение_:

In [12]:
g = (i for i in range(5))
g

<generator object <genexpr> at 0x7fea9f39a3b0>

Как видите, это не кортеж и не множество, а именно генератор.
Следовательно, никакого множества чисел от 0 до 5 пока нет.
Значит, и память компьютера ими не занята.
Однако всё также сработает протокол итератора для `g`:

In [13]:
for i in g:
    print(i, end=" ")

0 1 2 3 4 

Однако, мы можем использовать генераторы для конструирования списков, кортежей и колекций других типов.
Для этого достаточно передать генератор соответствующему конструктору:

In [14]:
# Список из генератора так (через списковое выражение)...
a = [i for i in range(5)]
print("a =", a)
# или так (через конструктор)
b = list(i for i in range(5))
print("b =", b)
# Кортеж из генератора (только через конструктор)
c = tuple(i for i in range(5))
print("c =", c)
# Множество также только через конструктор
d = set(i for i in range(5))
print("d =", d)

a = [0, 1, 2, 3, 4]
b = [0, 1, 2, 3, 4]
c = (0, 1, 2, 3, 4)
d = {0, 1, 2, 3, 4}


## Примеры генераторов

В данном разделе содержатся **примеры** использования генераторов.
В реальном коде изобретать велосипед не стоит - стандартные генераторы и функции работы с ними гораздо надёжнее и быстрее собственного решения.

### Собственная версия `range`

Мы почти сделали собственный `range`.
Сделаем его окончательно:

In [15]:
def range_gen(start=0, stop=None, step=1):
    i = start
    if stop is None:
        stop= start
        start = 0
    while i < stop:
        yield i
        i += step

# И пример его использования
print("range_gen:", end=" ")
for i in range_gen(3, 10, 2):
    print(i, end=" ")
# Сравните со стандартным range
print("\nrange:", end=" ")
for i in range(3, 10, 2):
    print(i, end=" ")

range_gen: 3 5 7 9 
range: 3 5 7 9 

На самом деле, кроме оператора `yield` существует ещё оператор (выражение) [`yield from`](https://docs-python.ru/tutorial/generatory-python/vyrazhenie-yield-from-expr/).
Его предназначение - связать два генератора.
Рассмотрим, что это значит на примере нашего `range_gen`:

In [16]:
def range_gen(start=0, stop=None, step=1):
    if stop is None:
        stop= start
        start = 0
    yield from range(start, stop, step)

Сравните с предыдущей версией и вы заметите, что теперь не нужен счётчик `i` и что наш генератор берёт значения из стандартного `range`.
Проверим работоспособность:

In [17]:
for i in range_gen(3, 10, 2):
    print(i, end=" ")

3 5 7 9 

Таким способом можно связать сколь-угодное число генераторов.

```{important}
Однако помните, `yield from` может запутать других и вас самих, если вы либо не хорошо понимаете предназначение данного выражения, либо если используете его неуместно.

Связывание генераторов без серьёзной на то причины усложняет код.
И тут полезно вспомнить [дзен Python](https://peps.python.org/pep-0020/): "Чем проще - тем лучше".
```

### Генератор бесконечной последовательности

Одним из преимуществ генераторов является их _ленивость_ - новое значение генератор выдаёт тогда и только тогда, когда оно запрашивается.
Генератор не создаёт никаких предварительных списков, что положительно сказывается на работе программы с оперативной памятью.
Более того, лень генераторов позволяет реализовать **бесконечную последовательность**:

In [18]:
def infinity():
    i = 0
    # Входим в бесконечный цикл
    while True:
        yield i
        i += 1

# Аналог бесконечного цикла 'while True',
# только счётчик 'i' получается сам собой
for i in infinity():
    if i == 5:
        # Условие выхода из цикла.
        # Не будь этого условия, числа выводились бы бесконечно
        break
    print(i, end=" ")

0 1 2 3 4 

Как это всё работает?
Рассмотрим на примере использования `infinity`:

1. В самом начале цикла (вернее, при его инициализации) вызывается генеративная функция `infinity`. В результате её вызова создан генератор. При этом поток выполнения программы вошёл в функцию `infinity`, создал локальную переменную `i = 0`, вошёл в бесконечный цикл `while True` и... вернулся в вызывающую функцию (в данном случае в основной поток программы) с объектом генератора в руках.
2. Цикл `for` начинаетотрабатывать протокол итератора. Применение `next` к генератору приводит к тому, что срабатывает `yield`, генерируя возвращаемое значение `i`, т.е. 0. Выполняется весь следующий код `infinity`: увеличивается значение `i += 1`, бесконечный цикл повторяется и вновь упирается в оператор `yield`... Поток выполнения `infinity` снова возвращает управление потоку-хозяину генератора (в данном случае основному потоку программы).
3. Выполняется тело цикла `for`.
4. Цикл `for` повторяется. При этом `i` уже равно 1. И так до бесконечности или пока цикл не будет прерван оператором `break`.

Если бы наш генератор был бы конечным, то применение `next` после достижения последнего значения привело бы к исключению `StopIteration`, и цикл `for` закончился бы (штатно).

Для прерывания цикла использовался оператор `break`.
Однако у генераторов, как и у любого объекта в Python, есть [свои методы](https://realpython.com/introduction-to-python-generators/).
Один из методов - `close` - предназначен для преждевременного закрытия генератора.
При этом генератор инициирует исключение `StopIteration`, из-за чего цикл автоматически прервётся.
Это позволяет в нашем случае использовать `close` вместо `break`:

```{note}
Обратите внимание на "оператор моржа" [`:=`](https://habr.com/ru/companies/skillfactory/articles/683418/).
```

```{attention}
Если вы итерируете генератор вне цикла `for`, например, в цикле `while` или вручную и закрываете его методом `close`, то вам следует перехватывать исключение `StopIteration`, иначе выполнение программы завершится с этой ошибкой.
Пример ниже.
```

In [20]:
g = infinity()
print(next(g))
print(next(g))
g.close()
# С этого момента не стоит использовать генератор
try:
    print(next(g))
except StopIteration:
    cprint("StopIteration:", "red", end=" ")
    print("да, инициировано исключение")
# Не будь try...except программа бы рухнула

0
1
StopIteration: да, инициировано исключение


### Собственная версия `zip`

In [21]:
# Наш zip тоже принимает произвольное число аргументов
def zip_gen(*sequences):
    try:
        # Заодно применим наш бесконечный генератор
        # так удобный здесь
        for i in infinity():
            # Генерируем кортеж из i-ых элементов sequences
            yield tuple(s[i] for s in sequences)
    # Условие выхода из бесконечного цикла - ошибка индексации.
    # Передаваемые списки могут иметь различные длины.
    except IndexError:
        # Да, генеративная функция обычная Python-функция,
        # поэтому можно вернуться из неё обычным return'ом.
        # Для генератора return сродни StopIteration.
        return

a = [1, 2, 3, 4]
b = [-2, -3]
c = [7, 0, 11]
# Наш zip
print("zip_gen:")
for i, j, k in zip_gen(a, b, c):
    print(i, j, k)
# Стандартный zip
print("zip:")
for i, j, k in zip(a, b, c):
    print(i, j, k)

zip_gen:
1 -2 7
2 -3 0
zip:
1 -2 7
2 -3 0


```{note}
Заметьте, мы не создали какие-либо дополнительные списки.
```

Таким образом, концепция генераторов в Python имеет широкие возможности.
При этом их синтаксис и логика действия предельно просты.
Представленной информации вполне достаточно, чтобы научиться работать с библиотекой SimPy.

## См. также

1. [Обучающий материал](https://realpython.com/introduction-to-python-generators/), подробно описывающий генераторы Python. Рассмотрены различные примеры их использования, а также особенности, не освещённые в данном справочнике. Например, методы генератора `send`, `close`, `throw`.
2. [Базовый материал](https://wiki.python.org/moin/Generators) по генераторам.